In [23]:
import numpy as np
import pandas as pd

In [69]:
positive = []
negative = []
ad = []
with open('../data/advertisment_positive.txt', encoding='utf-8') as file:
    for line in file:
        if line == "***\n":
            positive.append(ad)
            ad = []
        else:
            ad.append(line)


with open('../data/advertisment_negative.txt', encoding='utf-8') as file:
    for line in file:
        if line == "***\n":
            negative.append(ad)
            ad = []
        else:
            ad.append(line)


In [74]:
df_positive = pd.DataFrame({'text': positive})
df_negative = pd.DataFrame({'text': positive})

In [75]:
df_positive

,text
0,"[Opis ogłoszenia\n, SAMODZIELNA KSIĘGOWA\n, \n..."
1,"[Opis ogłoszenia\n, Manpower (Agencja zatrudni..."
2,"[Opis ogłoszenia\n, Zatrudnimy doracę, doradcz..."
3,"[Opis ogłoszenia\n, Jeśli:\n, \n, w pracy nie ..."
4,"[Opis ogłoszenia\n, Manpower (Agencja zatrudni..."
5,"[Opis ogłoszenia\n, Manpower (Agencja zatrudni..."
6,"[Opis ogłoszenia\n, Manpower (Agencja zatrudni..."
7,"[OPIS\n, Szukam ambitnych osób do zespołu, któ..."
8,"[OPIS\n, LUBISZ DORADZAĆ INNYM? RÓB TO Z NAMI!..."
9,"[OPIS\n, Doradca Klienta / Doradca finansowy\n..."
